In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 4

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [ ]:
# Drop the specified columns
df = df.drop(columns=['content'])

# Print the DataFrame to verify the changes
print(df.head())

   SubtitleID_index  subtitle_id  \
0             52663      9402655   
1             66004      9454096   
2             32167      9308944   
3             29417      9299035   
4             12977      9235190   

                                                name  \
0   american.dad.s05.e11.a.jones.for.a.smith.(2010).   
1      as.time.goes.by.s03.e06.branching.out.(1994).   
2     midsomer.murders.s07.e01.the.green.man.(2003).   
3  greys.anatomy.s19.e05.when.i.get.to.the.border...   
4                    klovn.s09.e04.alter.ego.(2022).   

                                    clean_text_lemma  \
0  Good morning, U.S.A. I got a feeling that it's...   
1  iYou must remember this,i iA kiss is still a k...   
2  Use the free code JOINNOW at www.playships.eu ...   
3  iWhen researcher studied the way surgeonsi ima...   
4  api.OpenSubtitles.org is deprecated, please im...   

                                    final_embeddings  
0  [0.1959221065044403, 0.18227329850196838, 0.40...  

In [ ]:
file_path = "/content/drive/MyDrive/content.xlsx"
dc = pd.read_excel(file_path)

In [ ]:
# Concatenate the two dataframes along the column axis
merged_df = pd.concat([df, dc], axis=1)

In [ ]:
def chunk_document(text, chunk_size=500, overlap=50):
    """Chunk a document into smaller parts with specified size and overlap."""
    if isinstance(text, float):
        text = str(text)
    chunks = []
    start_idx = 0
    end_idx = min(chunk_size, len(text))
    while start_idx < len(text):
        chunks.append(text[start_idx:end_idx])
        start_idx += chunk_size - overlap
        end_idx = min(start_idx + chunk_size, len(text))
    return chunks

def chunk_documents(merged_df, text_column='content', chunk_size=500, overlap=50):
    """Chunk the documents in a DataFrame."""
    merged_df['chunks'] = merged_df[text_column].apply(lambda x: chunk_document(x, chunk_size, overlap))
    return merged_df

# Chunk the documents in the DataFrame
chunk_size = 500  # Set your desired chunk size
overlap = 50  # Set the overlap size
chunked_df = chunk_documents(merged_df, text_column='clean_text_lemma', chunk_size=chunk_size, overlap=overlap)

# Display the first few rows of the chunked DataFrame
print(chunked_df.head())

   SubtitleID_index  subtitle_id  \
0             52663      9402655   
1             66004      9454096   
2             32167      9308944   
3             29417      9299035   
4             12977      9235190   

                                                name  \
0   american.dad.s05.e11.a.jones.for.a.smith.(2010).   
1      as.time.goes.by.s03.e06.branching.out.(1994).   
2     midsomer.murders.s07.e01.the.green.man.(2003).   
3  greys.anatomy.s19.e05.when.i.get.to.the.border...   
4                    klovn.s09.e04.alter.ego.(2022).   

                                    clean_text_lemma  \
0  Good morning, U.S.A. I got a feeling that it's...   
1  iYou must remember this,i iA kiss is still a k...   
2  Use the free code JOINNOW at www.playships.eu ...   
3  iWhen researcher studied the way surgeonsi ima...   
4  api.OpenSubtitles.org is deprecated, please im...   

                                    final_embeddings  \
0  [0.1959221065044403, 0.18227329850196838, 0.40... 

In [ ]:
print(chunked_df.tail())

       SubtitleID_index  subtitle_id  \
20647             74687      9489168   
20648             28698      9295776   
20649             79862      9510821   
20650              3263      9195935   
20651             41904      9351894   

                                                 name  \
20647             confessions.of.a.sexist.pig.(1998).   
20648      greys.anatomy.s12.e12.my.next.life.(2016).   
20649         criminal.minds.s01.e09.derailed.(2005).   
20650  big.love.s02.e04.rock.and.a.hard.place.(2007).   
20651               la.maniobra.de.la.tortuga.(2022).   

                                        clean_text_lemma  \
20647  INSTRUMENTAL MUSIC Um... Uh... Um... Uh... Bum...   
20648  iPreviously oni Grey's Anatomy... Goodbye, Der...   
20649  TELEPHONE RINGING MUTED CONVERSATIONS HEY. HEY...   
20650  Support u and become VIP member to remove all ...   
20651  Support u and become VIP member to remove all ...   

                                        final_embedding

In [ ]:

df['chunks_text_length'] = df['chunks'].apply(lambda x: len(x.split()))



,SubtitleID_index,subtitle_id,name,content,clean_text_lemma,text_length_lemma,final_embeddings,chunks,chunks_text_length
0,5,9180607,the.myth.(2005).eng.1cd,general the princesss convoy has entered our ...,general the princess convoy ha entered our rea...,4005,"[-0.1895383894443512, 0.1699831336736679, 0.49...",['general the princess convoy ha entered our r...,4499
1,6,9180608,the.great.beauty.(2013).eng.1cd,apiopensubtitlesorg is deprecated please impl...,apiopensubtitlesorg is deprecated please imple...,9208,"[-0.13504022359848022, 0.17723196744918823, 0....",['apiopensubtitlesorg is deprecated please imp...,6411
2,9,9180694,rudrabinar.obhishaap.s02.e03.anandagarher.akhh...,so youre assuming that my grandma mumtaz is ...,so youre assuming that my grandma mumtaz is ab...,1178,"[-0.13107889890670776, 0.1428164541721344, 0.2...",['so youre assuming that my grandma mumtaz is ...,1319
3,10,9180695,rudrabinar.obhishaap.s02.e04.udara.(2022).eng.1cd,you know that naads have less patience who s...,you know that naads have le patience who smugg...,1359,"[-0.08960047364234924, 0.04438433051109314, 0....",['you know that naads have le patience who smu...,1523
4,11,9180696,rudrabinar.obhishaap.s02.e05.saat.surer.mejaj....,use the free code joinnow at wwwplayshipseu ...,use the free code joinnow at wwwplayshipseu ge...,1017,"[-0.11988365650177002, 0.22249145805835724, 0....",['use the free code joinnow at wwwplayshipseu ...,1127


In [ ]:
!pip install ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.6 MB/s eta 0:00:00


In [ ]:
%load_ext sql
%sql sqlite:///my_chromadb.db

In [ ]:
file_path = ""
df = pd.read_excel(file_path)

In [ ]:
from openpyxl import Workbook

# Create a new Workbook object
wb = Workbook()

# Get the active worksheet
ws = wb.active

# Save the workbook to an Excel file
file_path = '/content/drive/My Drive/chunked_df.xlsx'
wb.save(file_path)

print("Empty Excel file created successfully.")


Empty Excel file created successfully.


In [ ]:
# Define the file path where you want to save the DataFrame
file_pathdf = '/content/drive/MyDrive/merged_df.xlsx'

# Save the DataFrame to an Excel file
chunked_df.to_excel(file_path, index=False)

In [ ]:
import sqlite3

# Specify the path to your desired location (e.g., /content/drive/MyDrive)
db_path = '/content/drive/MyDrive/ChromaDB.db'

# Connect to the SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect(db_path)
conn.close()  # Close the connection

print(f"SQLite database '{db_path}' created successfully.")


SQLite database '/content/drive/MyDrive/ChromaDB.db' created successfully.


In [ ]:
import pandas as pd
import sqlite3

# Load data from chunked.xlsx (assuming you've already loaded it into 'df')
file_pathdf = "/content/drive/MyDrive/chunked_df.xlsx"
df1 = pd.read_excel(file_pathdf)


In [ ]:
# Initialize SQLite database in the specified path
db_path = '/content/drive/MyDrive/ChromaDB.db'
conn = sqlite3.connect(db_path)

# Save DataFrame to SQLite
table_name = "my_subtitle_table"  # Choose an appropriate table name
df1.to_sql(table_name, conn, if_exists="replace", index=False)

print(f"DataFrame from {df1} is successfully saved to ChromaDB using SQLite: {db_path}")


DataFrame from        SubtitleID_index  subtitle_id  \
0                 52663      9402655   
1                 66004      9454096   
2                 32167      9308944   
3                 29417      9299035   
4                 12977      9235190   
...                 ...          ...   
20647             74687      9489168   
20648             28698      9295776   
20649             79862      9510821   
20650              3263      9195935   
20651             41904      9351894   

                                                    name  \
0       american.dad.s05.e11.a.jones.for.a.smith.(2010).   
1          as.time.goes.by.s03.e06.branching.out.(1994).   
2         midsomer.murders.s07.e01.the.green.man.(2003).   
3      greys.anatomy.s19.e05.when.i.get.to.the.border...   
4                        klovn.s09.e04.alter.ego.(2022).   
...                                                  ...   
20647                confessions.of.a.sexist.pig.(1998).   
20648         greys.anat

In [ ]:
# Initialize SQLite database in the specified path
db_path = '/content/drive/MyDrive/chromadb.db'
conn = sqlite3.connect(db_path)

# Save DataFrame to SQLite
df.to_sql("my_table_name", conn, if_exists="replace", index=False)

print(f"DataFrame from {file_path} is successfully saved to ChromaDB using SQLite: {db_path}")


In [ ]:
import sqlite3
import pandas as pd

# Path to the SQLite database file
db_file_path = '/content/drive/MyDrive/ChromaDB.db'

# Connect to the database
conn = sqlite3.connect(db_file_path)

# Query to select the 'num', 'name', and 'content' columns from the 'zipfiles' table
query = 'SELECT * FROM my_subtitle_table'

# Create a DataFrame from the query result
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the first few rows of the DataFrame
print(df.head())

   SubtitleID_index  subtitle_id  \
0             52663      9402655   
1             66004      9454096   
2             32167      9308944   
3             29417      9299035   
4             12977      9235190   

                                                name  \
0   american.dad.s05.e11.a.jones.for.a.smith.(2010).   
1      as.time.goes.by.s03.e06.branching.out.(1994).   
2     midsomer.murders.s07.e01.the.green.man.(2003).   
3  greys.anatomy.s19.e05.when.i.get.to.the.border...   
4                    klovn.s09.e04.alter.ego.(2022).   

                                    clean_text_lemma  \
0  Good morning, U.S.A. I got a feeling that it's...   
1  iYou must remember this,i iA kiss is still a k...   
2  Use the free code JOINNOW at www.playships.eu ...   
3  iWhen researcher studied the way surgeonsi ima...   
4  api.OpenSubtitles.org is deprecated, please im...   

                                    final_embeddings  \
0  [0.1959221065044403, 0.18227329850196838, 0.40... 

In [ ]:
import sqlite3

# Specify the path to your desired location (e.g., /content/drive/MyDrive)
db_path = '/content/drive/MyDrive/SE.db'

# Connect to the SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect(db_path)
conn.close()  # Close the connection

print(f"SQLite database '{db_path}' created successfully.")


SQLite database '/content/drive/MyDrive/SE.db' created successfully.


In [ ]:
# Initialize SQLite database in the specified path
db_path = '/content/drive/MyDrive/SE.db'
conn = sqlite3.connect(db_path)

# Save DataFrame to SQLite
table_name = "my_subtitle_table"  # Choose an appropriate table name
df.to_sql(table_name, conn, if_exists="replace", index=False)

print(f"DataFrame from {df} is successfully saved to SE using SQLite: {db_path}")

DataFrame from        SubtitleID_index  subtitle_id  \
0                     5      9180607   
1                     6      9180608   
2                     9      9180694   
3                    10      9180695   
4                    11      9180696   
...                 ...          ...   
20658             82482      9521866   
20659             82489      9521930   
20660             82490      9521931   
20661             82492      9521933   
20662             82495      9521938   

                                                    name  \
0                                the.myth.(2005).eng.1cd   
1                        the.great.beauty.(2013).eng.1cd   
2      rudrabinar.obhishaap.s02.e03.anandagarher.akhh...   
3      rudrabinar.obhishaap.s02.e04.udara.(2022).eng.1cd   
4      rudrabinar.obhishaap.s02.e05.saat.surer.mejaj....   
...                                                  ...   
20658  vanished.with.beth.holloway.s01.e08.fleischman...   
20659                   